# Example of RC beam optimal design

## Requirements

 - python3
 - cma python package: install via pip https://pypi.org/project/cma/#description


In [106]:
# problem: rectangular cross-section beam
b0 = .15 # thickness
h0 = .4  # height

rec = 0.03

L = 1.5 # length
    
gamma = 25e3 # specific weight of concrete
qsc = 10e3    # N/m load

fck = 25e6
fyk = 400e6


In [107]:
# import CMA-ES package
import cma
from math import sqrt

In [108]:
def structAnalysis(x):
    
    b = x[0]
    h = x[1]
    
    qpp = b * h * gamma
    print(qpp)
    Moment = (qsc + qpp) * L**2.0 / 8.0
    return Moment

In [109]:
def steelDesign(x,M):
    fcd = fck/1.5
    fyd = fyk/1.15
      
    b = x[0]
    h = x[1]
    d = h-rec

    Md   = M
    Mcrd = 0.251 * b * d**2 * fcd

    mud = Md/(b*d**2*fcd)
    
    omega_pos = (-1+sqrt(1-4*(-1/1.7)*(-mud)) ) / (2*(-1/1.7))
    omega_neg = (-1-sqrt(1-4*(-1/1.7)*(-mud)) ) / (2*(-1/1.7))
    
    #print("mud:", mud)
    #print("omegas:", omega_pos, " " , omega_neg)

    omega = omega_neg 
    if Md < Mcrd:
        
        Ast = omega * b*d*fcd / fyd
        Asc = 0.0
    else:
        DeltaM = Md - Mcrd
        Asc = DeltaM / ( (d-rec)*fyd )
        Ast = 0.306 * b * d * fcd / fyd + DeltaM / ( (d-rec)*fyd ) 
    return Ast, Asc

In [110]:
def costFun(x):
    
    M = structAnalysis(x)
    
    # compute steel area
    Ast, Asc = steelDesign( x, M)
    
    print("Steel: Ast: ", Ast , " Asc: ", Asc)

    # compute cost
    f = L * ( Ast + Asc )
    return f

In [111]:

x0 = [b0, h0]  # initial solution

M=structAnalysis(x0)

print("moment: ", M)

Ast,Asc = steelDesign(x0,M)
print("Steel: Ast: ", Ast , " Asc: ", Asc)

fun = costFun  # we could use `functools.partial(cma.ff.elli, cond=1e4)` to change the condition number to 1e4
sigma0 = .1    # initial standard deviation to sample new solutions
xopt, es = cma.fmin2(fun, x0, sigma0, {'bounds': [ [.1, .1], None]} )

In [112]:
dict(es.result._asdict())

In [113]:
es.plot();